In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.8 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 7.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 8.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 48.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 28.5 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.2 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-7pjdbabj/overlay/local/bin
    changing mode of /tmp/pip-build-env-7pjdbabj/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-7pjdbabj/overlay/lo

In [2]:
import re
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [3]:

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [4]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

In [5]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=1024, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [7]:
from google.colab import files
pd= files.upload()

Saving processed_reviews_128 (6).csv to processed_reviews_128 (6).csv


In [8]:
import pandas as pd
import random
df = pd.read_csv('processed_reviews_128 (6).csv')


In [9]:
df = df[['sentiment', 'cleaned_review']]
df.head()

,sentiment,cleaned_review
0,positive,mani funni scene peopl dont normal pay attent ...
1,negative,shame shame fine actor joseph fienn would allo...
2,negative,movi pretti predict nuff saidfrom delay kiss s...
3,positive,black mask jet li play bioengin superkil turn ...
4,negative,watch entir movi recogn particip william hurt ...


In [10]:
df_incorrect = df.loc[df['sentiment'] == 'negative']['cleaned_review'].reset_index(drop=True)
df_correct = df.loc[df['sentiment'] == 'positive']['cleaned_review'].reset_index(drop=True)
df_correct[random.choice(range(len(df_correct)))]

'know film blatantli aw cant help love well that evil ed possibl best aw film world sound rubbish dub crap screenplay nonsens special effect pap howev cant help love film dearli recommend least peopl year sam campbel guy play featur actor studio seri memor possibl greatest movi villain name toni montana serious dont expect lot wont disappoint keep lightheart approach watch film youll soon rate ten afterward'

In [11]:
df_incorrect[random.choice(range(len(df_incorrect)))]

'movi convolut plot contriv set frankli could follow surpris consid act dialogu could product kindergarten write like kathi ireland mayb youd want see movi probabl made vehicl tri get hollywood goal would say hope didnt invest much money product'

In [12]:
df_correct = 'peopl call comedi watch laughedbr br guess problem first saw br br wasnt old enough understand wasnt meant bebr br taken serious quit scene meantbr br funni care much charact laugh atbr br thembr br suggest watch film next time your fall lovebr br tri take serious think youll find despit sillybr br flaw one move love stori youv ever seen'
df_incorrect = 'read review decid rent dvd version br br like classic music want learn bach br br disappoint guess know enough bach music comment enough understand import music play br br mayb would appropri guidanc expert bach music explain film br br realli tri saw whole film hop would abl enjoy least br br see risk'

In [ ]:
#Step 6: Generating the Response

In [ ]:
prompts = []
model_responses = []

response_count = 0

while response_count < 20:
    prompt = f'''Examples of movie review ratings:
    1. {df_correct}
     ANSWER: positive
    2. {df_incorrect}
     ANSWER: negative

   Please print the answer from 0 to 1, where 1 - the review is more like positive, and 0 - the review is more like negative:
    "{df['cleaned_review'][random.choice(range(len(df)))][:128]}"
    '''

    prompt_template = f'''SYSTEM:You are a movie review evaluator. Your task is to print the answer (rating scale from 0 to 1).

    USER: {prompt}

    ASSISTANT:
    '''

    response = lcpp_llm(prompt=prompt_template, max_tokens=512, temperature=0.5, top_p=0.95,
                      repeat_penalty=1.2, top_k=150,
                      echo=True)

    print(response)
    print(response["choices"][0]["text"])
    response_text = response["choices"][0]["text"]
    response_text = "ASSISTANT: The probability of success is 0.85 which is a positive outcome."

    matches = re.findall(r'ASSISTANT:.*?(\b0(?:\.\d+)?|1(?:\.0+)?\b)', response_text, re.DOTALL)

    if matches:
        result = matches
        print(f"Model response: {result}")
        prompts.append(prompt_template)
        model_responses.append(result)
        response_count += 1
    else:
        print("Model response does not contain numbers between 0 and 1. Retrying...")

df_results = pd.DataFrame({
    'Prompt': prompts,
    'ModelResponse': model_responses
})





Llama.generate: prefix-match hit


In [38]:
df_results

,Prompt,ModelResponse
0,SYSTEM:You are a movie review evaluator. Your ...,[0.85]
1,SYSTEM:You are a movie review evaluator. Your ...,[0.85]


In [39]:
df_results.to_csv('responses.csv', index=False)

In [ ]:
!pip install scikit-learn

true_labels = [0,1,0,1,0,1,1,1,1,0]
predicted_labels = [1,1,1,0,1,1,1,1,1,1]

correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

accuracy = correct_predictions / len(true_labels)

print(f"Точность модели: {accuracy * 100:.2f}%")

Точность модели: 50.00%
